In [1]:
import xarray as xr
import rydanalysis as ra
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
import numpy as np
from tqdm.notebook import tqdm
from scipy.stats import sem
from scipy.integrate import quad

%matplotlib widget

In [2]:
path = Path.cwd().parent / "raw_data"

live_analysis_path = Path.cwd().parent / "Analysis" 
peaks_file = live_analysis_path / "peaks.h5"

In [3]:
data = ra.load_data(path, to_multiindex=True, lazy=None)
variable = data.ryd_data.variables.columns[0]
variable

2020-11-20 13:14:07.903 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


'PhaseShift'

In [ ]:
peak_df = data.scope_traces.peaks_summary.get_peak_description(width=2e-9, height=0.0007, distance=None)
peak_df.to_csv("peak_df.csv")

## Analysis

In [5]:
peak_df = pd.read_csv("peak_df.csv", index_col=[0, 1, 2])

In [6]:
summary = ra.summarize_peak_description(peak_df)
counts = summary.counts
counts_mean = counts.groupby(variable).mean()
counts_sem = counts.groupby(variable).reduce(sem)

In [7]:
fig, ax = plt.subplots()

ax.errorbar(counts_mean[variable], counts_mean, counts_sem, linestyle='-', marker='o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [10]:
cosine_model = ra.DampedCosineModel()
cosine_params = cosine_model.make_params()
cosine_params.add("phase", value=0, vary=True)
cosine_params.add("freq", value=1/360, vary=False, max=130, min=0.)
cosine_params.add("decay", value=0, vary=False, max=10, min=0)
cosine_params.add("amp", value=80, vary=True, max=300, min=0)
cosine_params.add("offset", value=50, vary=True, max=300, min=0)
fitres = cosine_model.fit(counts_mean, params=cosine_params, x=counts_mean[variable])#, weights=1/counts_sem)

fig, ax = plt.subplots()
ax.errorbar(counts_mean[variable], counts_mean, counts_sem, linestyle='', marker='o', label="counts")
x_values = np.linspace(counts_mean[variable].min(), counts_mean[variable].max(), 101)
ax.plot(x_values, fitres.eval(x=x_values))
times = np.linspace(0, 0.2, 110)
plt.text(0.15, 0.9,'freq: %.5g  $\pm$ %.5g phase: %.5g  $\pm$ %.5g'%(fitres.params['freq'].value, fitres.params['freq'].stderr, fitres.params['phase'].value, fitres.params['phase'].stderr), transform=ax.transAxes)
ax.set_xlabel(variable)
ax.set_ylabel("ion count")

plt.savefig('48P_%s_Fit_Cosine.png'%variable, dpi=600) # add leading zero to the figure name

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fitres